<div style="background-color:black;color:white;padding:20px">

<center><h3> LAA </h3></center>
<center><h1> Projeto Bicóptero </h1></center>

<center> <h1> Modelo dinâmico da bancada usando a formulação de Lagrange</h1></center>

<h3> Por Gabriel Henrique Costa e Silva </h3> 
</div>

<h2> Inicializando as bibliotecas </h2>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import * 
from sympy import trigsimp, pi, cos, sin

# Setup

First, import the necessary functions from SymPy that will allow us to construct time varying vectors in the four reference frames.

In [8]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point

SymPy has a rich printing system. Here we initialize printing so that all of the mathematical equations are rendered in standard mathematical notation.

In [9]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

The IPython notebook can display rich content. We will use the Image function to import some images for reference.

In [10]:
from IPython.display import Image

# Reference Frames and Orientation

Our first step is to define four reference frames, one each for the link 1, link 2, link 3 and link 4. These reference frames hold the information that defines how each frame is oriented relative to each other in addition to their relative angular velocites and angular accelerations. We start by creating four reference frames for each frame in the system.

In [11]:
inertial_frame = ReferenceFrame('I')

In [12]:
link_1_frame = ReferenceFrame('L1')

In [13]:
link_2_frame = ReferenceFrame('L2')

In [14]:
link_3_frame = ReferenceFrame('L3')

In [15]:
link_4_frame = ReferenceFrame('L4')

Now we need to specify how the frames are oriented with respect to each other. To do this, we need to define the three generalized coordinates, $\theta_1(t)$, $\theta_2(t)$, $\theta_3(t)$ and $\theta_4(t)$, angles which are time varying quantities.

In [16]:
t = Symbol("t")
theta1 = Function("theta1")(t)
theta2 = Function("theta2")(t)
theta3 = Function("theta3")(t)
theta4 = Function("theta4")(t)
g = dynamicsymbols('g')
fx, fy, fz = dynamicsymbols('f_x, f_y, f_z')
taux, tauy, tauz = dynamicsymbols('tau_x, tau_y, tau_z')

In [17]:
l1, l2, l3, l4 = dynamicsymbols('l1, l2, l3, l4')

In [18]:
m1, m2, m3, m4 = dynamicsymbols('m1, m2, m3, m4')

In [19]:
Ix1, Ix2, Ix3, Ix4 = dynamicsymbols('Ix1, Ix2, Ix3, Ix4')
Iy1, Iy2, Iy3, Iy4 = dynamicsymbols('Iy1, Iy2, Iy3, Iy4')
Iz1, Iz2, Iz3, Iz4 = dynamicsymbols('Iz1, Iz2, Iz3, Iz4')

In [20]:
# Criando a função de criação da matriz de transformação homogênea

# Matriz Rotação da Transformação
def rt_M(T):
    M = T[0:3,0:3]
    return M

In [140]:
# Matriz Rotação da Transformação
def ch_M(T):
    M=zeros(6)
    M[0:3,0:3] = T[0:3,0:3]
    M[3:6,3:6] = T[0:3,0:3]
    return M

In [21]:
# Matriz da massa e inércia
def mI_M(m,Ix,Iy,Iz):
    M = diag(m, m, m, Ix, Iy, Iz)
    return M

In [22]:
# Ponto de representação homogenea para dimensão 3
def ph_2N(ph):
    p = Matrix(ph[0:3])
    return (p)

In [23]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J.col_del(4)
    return J

In [24]:
# Definindo a nesima coluna da matriz jacobiana
def Ji(m0,op,oi):
    aux1 = m0*Matrix([0,0,1])
    aux2 = aux1.cross(op-oi)
    J = Matrix([aux2, aux1])
    return J

In [25]:
# Definindo a nesima coluna da matriz jacobiana
def JpTi(m0,op,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = m0*Matrix([0,0,1])
    oi = Matrix([Ti[0:3,3]])
    # Calculate the zi= i-1Mi.k
    aux2 = aux1.cross(op-oi)
    
    J = Matrix([aux2, aux1])
    return J

In [26]:
# Criando a função de criação da matriz de transformação homogênea
# a partir dos parâmetros de Denavit-Hatemberg

#cos = cos
#sin = sin

# Matriz Transformação Homogênea (Homogeous Transformation Matrix)
# Using the Standard Denavit-Hatemberg Convention (not Modified)
def ht_M(theta, d, alpha, a):
    # theta e alpha em radianos
    # d e a em metros
    Rz = Matrix([[cos(theta), -sin(theta), 0, 0],
                   [sin(theta), cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, a],
                   [0, 0, 0, 1]])
    ta = Matrix([[1, 0, 0, d],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = Matrix([[1, 0, 0, 0],
                   [0, cos(alpha), -sin(alpha), 0],
                   [0, sin(alpha), cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

<h2> Declaração dos parâmetros de Denavit-Hatenberg </h2>
<h2> Instanciação das matrizes de transformação homogêneas </h2>

In [27]:
T1 = ht_M(theta1, 0, (pi)/2, 0)
T2 = ht_M(theta2, l2, 0, 0)
T3 = ht_M(theta3, l3, (-pi)/2, 0)
T4 = ht_M(theta4, 0, 0, 0)

In [28]:
T1

Matrix([
[cos(theta1), 0,  sin(theta1), 0],
[sin(theta1), 0, -cos(theta1), 0],
[          0, 1,            0, 0],
[          0, 0,            0, 1]])

In [29]:
T2

Matrix([
[cos(theta2), -sin(theta2), 0, l2*cos(theta2)],
[sin(theta2),  cos(theta2), 0, l2*sin(theta2)],
[          0,            0, 1,              0],
[          0,            0, 0,              1]])

In [30]:
T3

Matrix([
[cos(theta3),  0, -sin(theta3), l3*cos(theta3)],
[sin(theta3),  0,  cos(theta3), l3*sin(theta3)],
[          0, -1,            0,              0],
[          0,  0,            0,              1]])

In [31]:
T4

Matrix([
[cos(theta4), -sin(theta4), 0, 0],
[sin(theta4),  cos(theta4), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

<h3> Simplificação da matriz T3 a partir da relação entre $\theta_3$ e $\theta_2$</h3>

In [32]:
T3

Matrix([
[cos(theta3),  0, -sin(theta3), l3*cos(theta3)],
[sin(theta3),  0,  cos(theta3), l3*sin(theta3)],
[          0, -1,            0,              0],
[          0,  0,            0,              1]])

In [33]:
T3=T3.subs(theta3,(-pi/2-theta2))

In [34]:
T3

Matrix([
[-sin(theta2),  0,  cos(theta2), -l3*sin(theta2)],
[-cos(theta2),  0, -sin(theta2), -l3*cos(theta2)],
[           0, -1,            0,               0],
[           0,  0,            0,               1]])

<h2> Instanciação das matrizes de transformação homogêneas em realação ao referencial 0 </h2>

In [35]:
T01=T1
T02=T01*T2
T02=trigsimp(T02)
T03=T02*T3
T03=trigsimp(T03)
T04=T03*T4

In [36]:
#(T02)
#(T03)
#(T04)

<h2> Criação das matrizes Jacobianas</h2>

In [37]:
# Criação dos pontos origens
og1=ph_2N(T01*Matrix([0,0,0,1]))
og2=ph_2N(T03*Matrix([-l3/2,0,0,1]))
og3=ph_2N(T03*Matrix([0,0,0,1]))
og4=ph_2N(T04*Matrix([0,0,0,1]))

In [39]:
# Calculo do Jc[1] que é o jacobiano referente ponto centro de gravidade g1
null=Matrix([0,0,0,0,0,0])
J1 = Ji(eye(3),og1, Matrix([0,0,0])) 
Jc1 = Jc(J1,null,null,null)
# Calculo do Jc[2] que é o jacobiano referente ponto centro de gravidade g2
J1 = Ji(eye(3),og2, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og2, Matrix([0,0,0])) 
Jc2 = trigsimp(Jc(J1,J2,null,null))
# Calculo do Jc[3] que é o jacobiano referente ponto centro de gravidade g3
J1 = Ji(eye(3),og3, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og3, Matrix([0,0,0]))
J3 = JpTi(rt_M(T02),og3,T02) 
Jc3 = trigsimp(Jc(J1,J2,J3,null))
# Calculo do Jc[4] que é o jacobiano referente ponto centro de gravidade g4
J1 = Ji(eye(3),og4, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og4, Matrix([0,0,0]))
J3 = JpTi(rt_M(T02),og4,T02) 
J4 = JpTi(rt_M(T03),og4,T03)
Jc4 = trigsimp(Jc(J1,J2,J3,J4))

In [40]:
Jc1

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[1, 0, 0, 0]])

In [41]:
Jc2

Matrix([
[-l2*sin(theta1)*cos(theta2), -(l2*sin(theta2) - l3/2)*cos(theta1), 0, 0],
[ l2*cos(theta1)*cos(theta2), -(l2*sin(theta2) - l3/2)*sin(theta1), 0, 0],
[                          0,                       l2*cos(theta2), 0, 0],
[                          0,                          sin(theta1), 0, 0],
[                          0,                         -cos(theta1), 0, 0],
[                          1,                                    0, 0, 0]])

In [42]:
Jc3

Matrix([
[-l2*sin(theta1)*cos(theta2), -(l2*sin(theta2) - l3)*cos(theta1), l3*cos(theta1), 0],
[ l2*cos(theta1)*cos(theta2), -(l2*sin(theta2) - l3)*sin(theta1), l3*sin(theta1), 0],
[                          0,                     l2*cos(theta2),              0, 0],
[                          0,                        sin(theta1),    sin(theta1), 0],
[                          0,                       -cos(theta1),   -cos(theta1), 0],
[                          1,                                  0,              0, 0]])

In [43]:
Jc4

Matrix([
[-l2*sin(theta1)*cos(theta2), -(l2*sin(theta2) - l3)*cos(theta1), l3*cos(theta1),           0],
[ l2*cos(theta1)*cos(theta2), -(l2*sin(theta2) - l3)*sin(theta1), l3*sin(theta1),           0],
[                          0,                     l2*cos(theta2),              0,           0],
[                          0,                        sin(theta1),    sin(theta1), cos(theta1)],
[                          0,                       -cos(theta1),   -cos(theta1), sin(theta1)],
[                          1,                                  0,              0,           0]])

In [44]:
# Calculo do J
J = Jc(J1,J2,J3,J4)

In [45]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J.col_del(4)
    return J

In [46]:
J

Matrix([
[-l2*sin(theta1)*cos(theta2),                            -(l2*sin(theta2) - l3)*cos(theta1), l3*cos(theta1),           0],
[ l2*cos(theta1)*cos(theta2),                            -(l2*sin(theta2) - l3)*sin(theta1), l3*sin(theta1),           0],
[                          0, l2*sin(theta1)**2*cos(theta2) + l2*cos(theta1)**2*cos(theta2),              0,           0],
[                          0,                                                   sin(theta1),    sin(theta1), cos(theta1)],
[                          0,                                                  -cos(theta1),   -cos(theta1), sin(theta1)],
[                          1,                                                             0,              0,           0]])

<h2> Criação das matrizes de Inércia</h2>

In [47]:
mI_1 = mI_M(m1,Ix1,Iy1,Iz1)
mI_2 = mI_M(m2,Ix2,Iy2,Iz2)
mI_3 = mI_M(m3,Ix3,Iy3,Iz3)
mI_4 = mI_M(m4,Ix4,Iy4,Iz4)
mI_4

Matrix([
[m4,  0,  0,   0,   0,   0],
[ 0, m4,  0,   0,   0,   0],
[ 0,  0, m4,   0,   0,   0],
[ 0,  0,  0, Ix4,   0,   0],
[ 0,  0,  0,   0, Iy4,   0],
[ 0,  0,  0,   0,   0, Iz4]])

In [48]:
# Build the Di matriz
def Di_M(Jc_gi,mI):
    M = (Jc_gi.T)*mI*Jc_gi;
    return M

In [49]:

D1 = Di_M(Jc1,mI_1)
D2 = trigsimp(Di_M(Jc2,mI_2))
D3 = trigsimp(Di_M(Jc3,mI_3))
D4 = trigsimp(Di_M(Jc4,mI_4))
Dtotal = D1+D2+D3+D4

In [50]:
Dtotal

Matrix([
[Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2*cos(theta2)**2 + l2**2*m3*cos(theta2)**2 + l2**2*m4*cos(theta2)**2,                                                                                                                                                                                                                                                                                          0,                                                                                                                                                                 0,                                       0],
[                                                                                                  0, Ix2*sin(theta1)**2 + Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy2*sin(theta1)**2 + Iy2 - Iy3*sin(theta1)**2 + Iy3 - Iy4*sin(theta1)**2 + Iy4 + l2**2*m2 + l2**2*m3 + l2**2*m4 - l2*l3*m2*sin(theta2) - 2*l2*l3*m3*sin(theta2) - 2*l2*l3*m4*sin(theta2) + l3**2*m2/4 + l3**2*m3 + l3**2*m4, Ix3*sin(theta1

<h2> Calculo da Energia Cinética </h2>

In [51]:
q=Matrix([theta1,theta2,theta3,theta4])
q_dot= diff(q,t)

In [52]:
K = 1/2* q_dot.T * Dtotal * q_dot
K

Matrix([[(0.5*(Ix4*cos(theta1)**2 + Iy4*sin(theta1)**2)*theta4' + 0.25*(Ix4 - Iy4)*sin(2*theta1)*theta2' + 0.25*(Ix4 - Iy4)*sin(2*theta1)*theta3')*theta4' + (0.25*(Ix4 - Iy4)*sin(2*theta1)*theta4' + 0.5*(Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy3*sin(theta1)**2 + Iy3 - Iy4*sin(theta1)**2 + Iy4 + l3**2*m3 + l3**2*m4)*theta3' + 0.5*(Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy3*sin(theta1)**2 + Iy3 - Iy4*sin(theta1)**2 + Iy4 - l2*l3*m3*sin(theta2) - l2*l3*m4*sin(theta2) + l3**2*m3 + l3**2*m4)*theta2')*theta3' + (0.25*(Ix4 - Iy4)*sin(2*theta1)*theta4' + 0.5*(Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy3*sin(theta1)**2 + Iy3 - Iy4*sin(theta1)**2 + Iy4 - l2*l3*m3*sin(theta2) - l2*l3*m4*sin(theta2) + l3**2*m3 + l3**2*m4)*theta3' + 0.5*(Ix2*sin(theta1)**2 + Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy2*sin(theta1)**2 + Iy2 - Iy3*sin(theta1)**2 + Iy3 - Iy4*sin(theta1)**2 + Iy4 + l2**2*m2 + l2**2*m3 + l2**2*m4 - l2*l3*m2*sin(theta2) - 2*l2*l3*m3*sin(theta2) - 2*l2*l3*m4*sin(theta2) + l3**2*m

<h2> Calculo da Energia Potencial</h2>

In [53]:
og1=T01*Matrix([0,0,0,1])
og2=T03*Matrix([-l3/2,0,0,1])
og3=T03*Matrix([0,0,0,1])
og4=T04*Matrix([0,0,0,1])

In [54]:
P1 = m1* g * Matrix([0,0,1,0]).T * og1
P2 = m2* g * Matrix([0,0,1,0]).T * og2
P3 = m3* g * Matrix([0,0,1,0]).T * og3
P4 = m4* g * Matrix([0,0,1,0]).T * og4

In [55]:
Ptotal = P1+ P2 + P3 + P4
Ptotal

Matrix([[(l2*sin(theta2) - l3)*g*m3 + (l2*sin(theta2) - l3)*g*m4 + (l2*sin(theta2) - l3/2)*g*m2]])

<h2> Calculo do Lagrangiano </h2>

In [56]:
L= K-Ptotal

<h2> Calculo dos termos da equaçoes de Euler Lagrange </h2>

In [57]:
# Termo D(q) contribuicao inercial vale D_total ja calculado precedemente
Dtotal

Matrix([
[Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2*cos(theta2)**2 + l2**2*m3*cos(theta2)**2 + l2**2*m4*cos(theta2)**2,                                                                                                                                                                                                                                                                                          0,                                                                                                                                                                 0,                                       0],
[                                                                                                  0, Ix2*sin(theta1)**2 + Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy2*sin(theta1)**2 + Iy2 - Iy3*sin(theta1)**2 + Iy3 - Iy4*sin(theta1)**2 + Iy4 + l2**2*m2 + l2**2*m3 + l2**2*m4 - l2*l3*m2*sin(theta2) - 2*l2*l3*m3*sin(theta2) - 2*l2*l3*m4*sin(theta2) + l3**2*m2/4 + l3**2*m3 + l3**2*m4, Ix3*sin(theta1

In [58]:
q[1]
diff(Dtotal,q[1])*q_dot[1]

Matrix([
[(-2*l2**2*m2*sin(theta2)*cos(theta2) - 2*l2**2*m3*sin(theta2)*cos(theta2) - 2*l2**2*m4*sin(theta2)*cos(theta2))*theta2',                                                                                 0,                                                      0, 0],
[                                                                                                                      0, (-l2*l3*m2*cos(theta2) - 2*l2*l3*m3*cos(theta2) - 2*l2*l3*m4*cos(theta2))*theta2', (-l2*l3*m3*cos(theta2) - l2*l3*m4*cos(theta2))*theta2', 0],
[                                                                                                                      0,                            (-l2*l3*m3*cos(theta2) - l2*l3*m4*cos(theta2))*theta2',                                                      0, 0],
[                                                                                                                      0,                                                                           

In [59]:
def C_t(q,q_dot,D_total):
    C_1 = zeros(4,4)
    C_2 = zeros(4,1)

    for n in range(4):
        # Part 1 = dD(q)/dt = sum (partial D(q)/partial qi)q_dot
        C_1= C_1 + (diff(D_total,q[n])*q_dot[n])

    C_1 = C_1*q_dot

    for n in range(4):
        # Part 2 = column matrix (1/2)*(q_dot)^T.(partial D(q)/partial qi).q_do
        C_2[n] = (1/2)*((q_dot.T)*diff(D_total,q[n])*q_dot)

    C = C_1 - C_2;
    return C

In [60]:
# Termo C(q,qdot) contribuicao inercial

Ctotal = C_t(q, q_dot, Dtotal)
Ctotal

Matrix([
[-0.5*((-2*Ix4*sin(theta1)*cos(theta1) + 2*Iy4*sin(theta1)*cos(theta1))*theta4' + 2*(Ix4/2 - Iy4/2)*cos(2*theta1)*theta2' + 2*(Ix4/2 - Iy4/2)*cos(2*theta1)*theta3')*theta4' - 0.5*(2*(Ix4/2 - Iy4/2)*cos(2*theta1)*theta4' + (2*Ix3*sin(theta1)*cos(theta1) + 2*Ix4*sin(theta1)*cos(theta1) - 2*Iy3*sin(theta1)*cos(theta1) - 2*Iy4*sin(theta1)*cos(theta1))*theta2' + (2*Ix3*sin(theta1)*cos(theta1) + 2*Ix4*sin(theta1)*cos(theta1) - 2*Iy3*sin(theta1)*cos(theta1) - 2*Iy4*sin(theta1)*cos(theta1))*theta3')*theta3' - 0.5*(2*(Ix4/2 - Iy4/2)*cos(2*theta1)*theta4' + (2*Ix3*sin(theta1)*cos(theta1) + 2*Ix4*sin(theta1)*cos(theta1) - 2*Iy3*sin(theta1)*cos(theta1) - 2*Iy4*sin(theta1)*cos(theta1))*theta3' + (2*Ix2*sin(theta1)*cos(theta1) + 2*Ix3*sin(theta1)*cos(theta1) + 2*Ix4*sin(theta1)*cos(theta1) - 2*Iy2*sin(theta1)*cos(theta1) - 2*Iy3*sin(theta1)*cos(theta1) - 2*Iy4*sin(theta1)*cos(theta1))*theta2')*theta2' + (-2*l2**2*m2*sin(theta2)*cos(theta2) - 2*l2**2*m3*sin(theta2)*cos(theta2) - 2*l2**2*m4*s

In [61]:
def G_t(q,P_total):
    G = zeros(4,1)
    for n in range(4):
        G[n] = diff(P_total,q[n])
    return G

In [62]:
# Termo G(q)  vale as contribuicoes da gravidade 
Gtotal = G_t(q,Ptotal)
Gtotal

Matrix([
[                                                              0],
[g*l2*m2*cos(theta2) + g*l2*m3*cos(theta2) + g*l2*m4*cos(theta2)],
[                                                              0],
[                                                              0]])

Calculo do esforcos de interaçao Gamma_e  



Representar os tensores cinematicos em relacao a P
1-Identificar o vetores OiP sendo P o ponto do referencial bicoptero
2-Multiplicar por i-1Mi (matriz de rotacao) 
3-fazer o produto vetorial entre Zi x (i-1Mi * OiP)
4-Montar o tensor


In [63]:
def v_i(M0,Tp,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = M0*Matrix([0,0,1])

    Op = Tp[0:3,3]
    Oi = Ti[0:3,3]

    # Cross Product  zi x (Op-Oi)
    aux2 = (aux1.cross(Op-Oi))
    
    # Building cinematic tensor for pivot joints
    
    v = Matrix([aux2, aux1])
    return v 

In [64]:
#Matriz rotacional 0^M_0
# Calculate the zi= i-1Mi.k 
J1 = v_i(eye(3), T04, zeros(4,4))

J2 = trigsimp(v_i(rt_M(T01), T04, T01))

J3 = v_i(rt_M(T02),T04,T02)

J4 = v_i(rt_M(T03),T04,T03)

JcP4 = Jc(J1,J2,J3,J4)

v = (JcP4*Matrix([0,1,-1,0])).subs({theta1: 0, theta2:0,theta4: 0})
#(JcP4.T)
#v
#(JcP4.T)*Matrix([-fx,0,0,0,0,0])


In [65]:
JcP4

Matrix([
[-l2*sin(theta1)*cos(theta2), -(l2*sin(theta2) - l3)*cos(theta1), l3*cos(theta1),           0],
[ l2*cos(theta1)*cos(theta2), -(l2*sin(theta2) - l3)*sin(theta1), l3*sin(theta1),           0],
[                          0,                     l2*cos(theta2),              0,           0],
[                          0,                        sin(theta1),    sin(theta1), cos(theta1)],
[                          0,                       -cos(theta1),   -cos(theta1), sin(theta1)],
[                          1,                                  0,              0,           0]])

Calcular o Gamma_e a partir da matriz Jacobiana cinematica em relacao a P

In [66]:
#Forca expressa no referencial 0 (Não é o que queremos)

#tau_e:=<<f[x],f[y],f[z],tau[x],tau[y],tau[z]>>:
tau_e = Matrix([fx,fy,0,0,0,tauz])
Gamma_e = JcP4.T*(tau_e)
Gamma_e.subs({theta2:0,theta1:0,theta4:0})


Matrix([
[f_y*l2 + tau_z],
[        f_x*l3],
[        f_x*l3],
[             0]])

In [121]:
# Para entao representar o vetor esforco em outro referencial é necessario as seguintes formulacoes

# Para entender indica-se o exemplo da maçã do link https://modernrobotics.northwestern.edu/nu-gm-book-resource/3-4-wrenches/#department
#Ttest:=eval(ht_M(0,-(ele),0,0));
#tau_e:=<<0,0,0,0,-mg,0>>;
#p:=v_2so3(<<Ttest(1,4),Ttest(2,4),Ttest(3,4)>>);
#T_adj(Ttest,rt_M(Ttest));
#tau_f:=LinearAlgebra[Transpose](T_adj(Ttest)).(tau_e);


#representacao de um vetor em so3
def v_2so3(v):
    p = zeros(3)
    p[0,1] = -v[2,0]
    p[0,2] = v[1,0]
    p[1,2] = -v[0,0]
    p[1,0] = v[2,0]
    p[2,0] = -v[1,0]
    p[2,1] = v[0,0]
    return p

#representacao adjunta da matriz Transformacao homogenea
def T_adj(Thom):
    p = v_2so3(Thom[0:3,3])

    K = p*rt_M(Thom)
    M = zeros(6)

    M[0:3,0:3] = Thom[0:3,0:3]
    M[3:6,0:6] = Matrix([K[0:3,0:3]]).col_insert(3,Thom[0:3,0:3]) 

    return M

#Inversa da matriz homogênea
def inv_Thom(Thom):
    Ka = (rt_M(Thom)).T
    Ke = -Ka*Thom[0:3,3] 

    M = Ka[0:3,0:3]
    M = M.col_insert(3,Ke[0:3,0])

    M = M.row_insert(4, Matrix([0,0,0,1]).T)
    return M

In [128]:
#Calcula-se a matrix inversa (T_0_4) da matriz transformacao homogenea T_04 
T40 =inv_Thom(T04)

#eval(T_4_[0],{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});

#Passa-se o vetor 3x1 p da entao matriz inversa para s03
p=v_2so3(T40[0:3,3])

#Finalmente calcula-se a matrix adjunta de T_4_0
MAdjT40 = T_adj(T40)
#T40
#p
#MAdjT40
#eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});

#LinearAlgebra[Transpose](eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0})).<<0,0,tau[z],f[x],f[y],0>> := LinearAlgebra[Transpose](eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0})).<<0,0,tau[z],f[x],f[y],0>> ;

#Constroi-se uma matrix capaz de colocar os elementos momentos nas ultimas 3 posicoes do vetor 6x1 e colocar as forcas nas 3 primeiras posicoes do vetor
MF_M = zeros(6)
MF_M[0,3] =1
MF_M[1,4] =1
MF_M[2,5] =1
MF_M[3,0] =1
MF_M[4,1] =1
MF_M[5,2] =1


Matrix([
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1],
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0]])

In [145]:
#Perceba que inicialmente o vetor tau_e é da maneira <<momentos, forcas>>
# Alem disso para o nosso aplicacao taux=tauy=fz=0
tau_e = Matrix([0,0,tauz,fx,fy,0])

#Aqui mostra-se como os esforcos em P sao traduzidos no referencial 0
#eval(MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6),{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0}):
PJ4 = trigsimp(ch_M(rt_M(T40))*JcP4);

#aux:=MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6):

#LinearAlgebra[Transpose](J_cP[4]);

#Calcula-se Gamma_e
#Gamma_e:=LinearAlgebra[Transpose](J_cP[4]).MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6):

Gamma_e = PJ4.T*MF_M*(tau_e)

#Nosso caso queremos Gamma_b
Gamma_b = Gamma_e

Gamma_b = trigsimp(Gamma_b)
Gamma_b
#Visualização dos esforços em cada junta na posicao neutra 
#MF_M.<<0,0,tau[z],f[x],f[y],0>>;
#eval(MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6),{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});
#eval(Gamma_b,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});


Matrix([
[ (f_x*sin(theta4) + f_y*cos(theta4))*l2*cos(theta2)],
[(-f_x*cos(theta4) + f_y*sin(theta4))*l2*cos(theta2)],
[                                                  0],
[                                              tau_z]])

In [148]:
q=Matrix([theta1,theta2,theta3,theta4])
q_dot= diff(q,t)
q_ddot= diff(q_dot,t)

Matrix([
[theta1''],
[theta2''],
[theta3''],
[theta4'']])